# Data Download & Processing

The objective of this script is to create a common dataset for our analysis. It includes the following functions:
- download MTA bus data from May 1 to 31, 2017
- process data in each file 
- select only M15 and M15 SBS buses
- concatenate daily dataset into one consolidated dataset

---

# Download MTA Bus Data

- Download
- Unzip
- Remove zipped files

In [4]:
# Download Bus Time for all May, 2017
# Only do it at set up

'''
for i in range(1, 32):
    
    if i < 10: 
        file = 'bus_time_2017050' + str(i) + '.csv.xz'
    else: 
        file = 'bus_time_201705' + str(i) + '.csv.xz'
    
    url = 'http://data.mytransit.nyc.s3.amazonaws.com/bus_time/2017/2017-05/' + file 
    
    print("--- Downloading MTA Bus Data of May %i, 2017 ---" %i)
    !wget -nc $url -P './download'
    
    print("--- Unzipping MTA Bus Data of May %i, 2017 ---" %i)
    path = "./download/" + file
    !7za e $path -o./data
'''

--- Downloading MTA Bus Data of May 10, 2017 ---
--2017-11-21 17:46:13--  http://data.mytransit.nyc.s3.amazonaws.com/bus_time/2017/2017-05/bus_time_20170510.csv.xz
Resolving data.mytransit.nyc.s3.amazonaws.com... 54.231.120.59
Connecting to data.mytransit.nyc.s3.amazonaws.com|54.231.120.59|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47802948 (46M) [text/csv]
Saving to: “./download/bus_time_20170510.csv.xz”

100%[======================================>] 47,802,948  46.6M/s   in 1.0s    

2017-11-21 17:46:14 (46.6 MB/s) - “./download/bus_time_20170510.csv.xz” saved [47802948/47802948]

--- Unzipping MTA Bus Data of May 10, 2017 ---

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,64 CPUs Intel(R) Xeon(R) CPU E5-4640 0 @ 2.40GHz (206D7),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan ./download                    1 file, 47802948 bytes (46 MiB)

Extracting arc

--2017-11-21 17:46:34--  http://data.mytransit.nyc.s3.amazonaws.com/bus_time/2017/2017-05/bus_time_20170513.csv.xz
Resolving data.mytransit.nyc.s3.amazonaws.com... 52.216.96.115
Connecting to data.mytransit.nyc.s3.amazonaws.com|52.216.96.115|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32768580 (31M) [text/csv]
Saving to: “./download/bus_time_20170513.csv.xz”

100%[======================================>] 32,768,580  60.5M/s   in 0.5s    

2017-11-21 17:46:35 (60.5 MB/s) - “./download/bus_time_20170513.csv.xz” saved [32768580/32768580]

--- Unzipping MTA Bus Data of May 13, 2017 ---

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,64 CPUs Intel(R) Xeon(R) CPU E5-4640 0 @ 2.40GHz (206D7),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan ./download                    1 file, 32768580 bytes (32 MiB)

Extracting archive: ./download/bus_time_20170513.csv.xz
--
Path

      3% - bus_time_20170516.c                            7% - bus_time_20170516.c                           10% - bus_time_20170516.c                           13% - bus_time_20170516.c                           17% - bus_time_20170516.c                           20% - bus_time_20170516.c                           24% - bus_time_20170516.c                           28% - bus_time_20170516.c                           31% - bus_time_20170516.c                           35% - bus_time_20170516.c                           39% - bus_time_20170516.c                           42% - bus_time_20170516.c                           46% - bus_time_20170516.c                           49% - bus_time_20170516.c                           53% - bus_time_20170516.c                           56% - bus_time_20170516.c                           60% - bus_time_20170516.c                           64% - bus_time_20170516.c                           67% - bus_time_20170516.c                           71% - b

      3% - bus_time_20170519.c                            6% - bus_time_20170519.c                           10% - bus_time_20170519.c                           13% - bus_time_20170519.c                           16% - bus_time_20170519.c                           20% - bus_time_20170519.c                           23% - bus_time_20170519.c                           27% - bus_time_20170519.c                           30% - bus_time_20170519.c                           34% - bus_time_20170519.c                           37% - bus_time_20170519.c                           41% - bus_time_20170519.c                           44% - bus_time_20170519.c                           48% - bus_time_20170519.c                           51% - bus_time_20170519.c                           55% - bus_time_20170519.c                           58% - bus_time_20170519.c                           61% - bus_time_20170519.c                           65% - bus_time_20170519.c                           68% - b

      3% - bus_time_20170522.c                            6% - bus_time_20170522.c                           10% - bus_time_20170522.c                           13% - bus_time_20170522.c                           17% - bus_time_20170522.c                           21% - bus_time_20170522.c                           24% - bus_time_20170522.c                           28% - bus_time_20170522.c                           31% - bus_time_20170522.c                           35% - bus_time_20170522.c                           39% - bus_time_20170522.c                           43% - bus_time_20170522.c                           47% - bus_time_20170522.c                           51% - bus_time_20170522.c                           55% - bus_time_20170522.c                           59% - bus_time_20170522.c                           63% - bus_time_20170522.c                           68% - bus_time_20170522.c                           72% - bus_time_20170522.c                           76% - b

      3% - bus_time_20170525.c                            6% - bus_time_20170525.c                           10% - bus_time_20170525.c                           13% - bus_time_20170525.c                           17% - bus_time_20170525.c                           20% - bus_time_20170525.c                           24% - bus_time_20170525.c                           27% - bus_time_20170525.c                           31% - bus_time_20170525.c                           34% - bus_time_20170525.c                           38% - bus_time_20170525.c                           41% - bus_time_20170525.c                           45% - bus_time_20170525.c                           48% - bus_time_20170525.c                           52% - bus_time_20170525.c                           55% - bus_time_20170525.c                           59% - bus_time_20170525.c                           62% - bus_time_20170525.c                           66% - bus_time_20170525.c                           69% - b

      5% - bus_time_20170528.c                           10% - bus_time_20170528.c                           16% - bus_time_20170528.c                           21% - bus_time_20170528.c                           26% - bus_time_20170528.c                           32% - bus_time_20170528.c                           37% - bus_time_20170528.c                           43% - bus_time_20170528.c                           49% - bus_time_20170528.c                           54% - bus_time_20170528.c                           60% - bus_time_20170528.c                           65% - bus_time_20170528.c                           71% - bus_time_20170528.c                           76% - bus_time_20170528.c                           82% - bus_time_20170528.c                           88% - bus_time_20170528.c                           93% - bus_time_20170528.c                           99% - bus_time_20170528.c                          Everything is Ok

Size:       241438097
Compressed: 27843240

      2% - bus_time_20170531.c                            6% - bus_time_20170531.c                            9% - bus_time_20170531.c                           12% - bus_time_20170531.c                           15% - bus_time_20170531.c                           18% - bus_time_20170531.c                           22% - bus_time_20170531.c                           25% - bus_time_20170531.c                           28% - bus_time_20170531.c                           32% - bus_time_20170531.c                           35% - bus_time_20170531.c                           38% - bus_time_20170531.c                           41% - bus_time_20170531.c                           45% - bus_time_20170531.c                           48% - bus_time_20170531.c                           51% - bus_time_20170531.c                           54% - bus_time_20170531.c                           58% - bus_time_20170531.c                           61% - bus_time_20170531.c                           64% - b

In [43]:
file = 'http://data.mytransit.nyc.s3.amazonaws.com/gtfs/2015/gtfs_nyct_bus_20150103.zip'
!wget -nc $file -P './download'

--2017-11-21 18:50:54--  http://data.mytransit.nyc.s3.amazonaws.com/gtfs/2015/gtfs_nyct_bus_20150103.zip
Resolving data.mytransit.nyc.s3.amazonaws.com... 52.216.0.16
Connecting to data.mytransit.nyc.s3.amazonaws.com|52.216.0.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94088821 (90M) [application/zip]
Saving to: “./download/gtfs_nyct_bus_20150103.zip”

100%[======================================>] 94,088,821  36.7M/s   in 2.4s    

2017-11-21 18:50:57 (36.7 MB/s) - “./download/gtfs_nyct_bus_20150103.zip” saved [94088821/94088821]



In [44]:
path = './download/gtfs_nyct_bus_20150103.zip'
!7za e $path -o./data


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,64 CPUs Intel(R) Xeon(R) CPU E5-4640 0 @ 2.40GHz (206D7),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan ./download                    1 file, 94088821 bytes (90 MiB)

Extracting archive: ./download/gtfs_nyct_bus_20150103.zip
--
Path = ./download/gtfs_nyct_bus_20150103.zip
Type = zip
Physical Size = 94088821

      2% 5 - shapes.tx                    6% 7 - stop_times.tx                        9% 7 - stop_times.tx                       12% 7 - stop_times.tx                       16% 7 - stop_times.tx                       19% 7 - stop_times.tx                       23% 7 - stop_times.tx                       27% 7 - stop_times.tx                       31% 7 - stop_times.tx                       34% 7 - stop_times.tx                       37% 7 - stop_times.tx                       40% 7 - stop_times.tx                       43% 7 - stop_

In [24]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame()
total_df = pd.DataFrame()

In [45]:
trips = pd.read_csv("./data/trips.txt")
stops = pd.read_csv("./data/stops.txt")
schedules = pd.read_csv("./data/stop_times.txt")

In [53]:
for i in range(1, 4):
    try:
        if i < 10: 
            file = './data/bus_time_2017050' + str(i) + '.csv'
        else: 
            file = './data/bus_time_201705' + str(i) + '.csv'

        # ingest into a data frame
        print(" --- Processing %s ---" %file)
        df = pd.read_csv(file)

        # convert timestamp to DateTime
        df['timestamp'] = pd.to_datetime(df['timestamp'], infer_datetime_format = True)
        df['Day'] = df['timestamp'].apply(lambda x: x.day)
        df['Hour'] =df['timestamp'].apply(lambda x: x.hour)
        df['DOW'] = df['timestamp'].dt.dayofweek

        days = {0:'Mon',1:'Tues',2:'Weds',3:'Thurs',4:'Fri',5:'Sat',6:'Sun'}
        df['DOW_Label'] = df['DOW'].apply(lambda x: days[x])
        
        print(" --- Done (1/3) ---")
        
        #throw away trips with <15 reports
        good_trips_only = df.groupby(df.trip_id).filter(lambda group: len(group) > 15)
        
        print(" --- Done (2/3) --- ")
        
        #concatenate 
    
        total_df = pd.concat([total_df, df])
        
        print(" --- Done (3/3) --- ")
        
    except Exception as ex:
        print(ex)
        pass

 --- Processing ./data/bus_time_20170501.csv ---


/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


 --- Done (1/3) ---
 --- Done (2/3) --- 
 --- Done (3/3) --- 
 --- Processing ./data/bus_time_20170502.csv ---
 --- Done (1/3) ---
 --- Done (2/3) --- 
 --- Done (3/3) --- 
 --- Processing ./data/bus_time_20170503.csv ---
 --- Done (1/3) ---
 --- Done (2/3) --- 
 --- Done (3/3) --- 


In [54]:
total_df.shape
#m15_only = df[df.route_id.str.contains("M15")].reset_index()

(12661727, 16)

In [11]:
total_df = pd.DataFrame()
total_df = pd.concat([total_df, df])

(3205831, 14)